<a href="https://colab.research.google.com/github/WoradeeKongthong/medical_cost_regression/blob/master/02_Medical_Cost_OLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
## Data Preprocessing

# importing the dataset
df = pd.read_csv('/content/drive/My Drive/life-long learning/MyProjects/Medical Cost/cleaned_insurance.csv')

In [3]:
df.head()   

,age,sex,bmi,children,smoker,region,charges,bmi_status,AgeInterval
0,19,female,27.900,0,True,southwest,16884.92400,overweight,"(15, 25]"
1,18,male,33.770,1,False,southeast,1725.55230,obesity,"(15, 25]"
2,28,male,33.000,3,False,southeast,4449.46200,obesity,"(25, 45]"
3,33,male,22.705,0,False,northwest,21984.47061,ideal,"(25, 45]"
4,32,male,28.880,0,False,northwest,3866.85520,overweight,"(25, 45]"


In [0]:
X = df.iloc[:, :6].values
y = df.iloc[:, 6].values

In [5]:
print(X)

[[19 'female' 27.9 0 True 'southwest']
 [18 'male' 33.77 1 False 'southeast']
 [28 'male' 33.0 3 False 'southeast']
 ...
 [18 'female' 36.85 0 False 'southeast']
 [21 'female' 25.8 0 False 'southwest']
 [61 'female' 29.07 0 True 'northwest']]


In [6]:
print(y)

[16884.924   1725.5523  4449.462  ...  1629.8335  2007.945  29141.3603]


In [0]:
# encoding categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1,4,5])], remainder='passthrough')
X = ct.fit_transform(X)

In [8]:
print(X[:5])

[[0.0 1.0 0.0 0.0 1.0 19 27.9 0]
 [1.0 0.0 0.0 1.0 0.0 18 33.77 1]
 [1.0 0.0 0.0 1.0 0.0 28 33.0 3]
 [1.0 0.0 1.0 0.0 0.0 33 22.705 0]
 [1.0 0.0 1.0 0.0 0.0 32 28.88 0]]


In [0]:
# Implementing automated Backward Elimination with Adjusted R-square
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
X = add_constant(X)
X = X.astype(float)          

In [0]:
def backwardElimination(X,y,sl):
  global X_cat_track
  X_cat_track = np.arange(len(X[0]))

  for i in range(0, len(X[0])):
    
    regressor_OLS = OLS(y, X).fit()
    maxPval = max(regressor_OLS.pvalues).astype(float)
    maxParg = np.argmax(regressor_OLS.pvalues)
    adjR2 = regressor_OLS.rsquared_adj.astype(float)
    
    if maxPval > sl :
      # try backward elimination
      X_temp = X
      X_temp = np.delete(X_temp, maxParg, 1)
      regressor_OLS_temp = OLS(y, X_temp).fit()
      adjR2_temp = regressor_OLS_temp.rsquared_adj.astype(float)

      if adjR2 >= adjR2_temp:
        # not apply backward elimination
        print(regressor_OLS.summary())
        return (X, regressor_OLS)
      else:
        # apply backward elimination and continue
        X = np.delete(X, maxParg, 1)
        X_cat_track = np.delete(X_cat_track, maxParg,0)
    else:
      print(regressor_OLS.summary())
      return (X, regressor_OLS)

In [39]:
X_opt, regressor_OLS = backwardElimination(X,y,0.05)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     668.3
Date:                Sun, 10 May 2020   Prob (F-statistic):               0.00
Time:                        05:17:14   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1331   BIC:                         2.715e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.217e+04    949.538    -12.812      0.0

In [40]:
# figuring out remaining components of X

enc_group = len(ct.named_transformers_['encoder'].categories_)
X_categories = ['constant']
for i in range(enc_group):
  catielem = len(ct.named_transformers_['encoder'].categories_[i])
  for j in range(1,catielem):
    X_categories.append(ct.named_transformers_['encoder'].categories_[i][j])
X_categories.extend(['age','bmi','children'])
X_categories = np.array(X_categories)
X_categories

X_categories[X_cat_track.tolist()]

array(['constant', 'True', 'southeast', 'southwest', 'age', 'bmi',
       'children'], dtype='<U9')